##  Binary Classification using Perception, MLP and Autoencoder

#### Loading the required libraries

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc

from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

import matplotlib.pyplot as plt

C:\Users\keerthi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
import os
os.chdir("F:\\predict-bo-trial")

#### Problem

    Is to identify products at risk of backorder before the event occurs so the business has time to react. 

#### Data

Data file contains the historical data for the 8 weeks prior to the week we are trying to predict. The data was taken as weekly snapshots at the start of each week. Columns are defined as follows:

    sku - Random ID for the product

    national_inv - Current inventory level for the part

    lead_time - Transit time for product (if available)

    in_transit_qty - Amount of product in transit from source

    forecast_3_month - Forecast sales for the next 3 months

    forecast_6_month - Forecast sales for the next 6 months

    forecast_9_month - Forecast sales for the next 9 months

    sales_1_month - Sales quantity for the prior 1 month time period

    sales_3_month - Sales quantity for the prior 3 month time period

    sales_6_month - Sales quantity for the prior 6 month time period

    sales_9_month - Sales quantity for the prior 9 month time period

    min_bank - Minimum recommend amount to stock

    potential_issue - Source issue for part identified

    pieces_past_due - Parts overdue from source

    perf_6_month_avg - Source performance for prior 6 month period

    perf_12_month_avg - Source performance for prior 12 month period

    local_bo_qty - Amount of stock orders overdue

    deck_risk - Part risk flag

    oe_constraint - Part risk flag

    ppap_risk - Part risk flag

    stop_auto_buy - Part risk flag

    rev_stop - Part risk flag

    went_on_backorder - Product actually went on backorder. This is the target value.

#### Loading the data

In [0]:
data = pd.read_csv("strain.csv",header=0)


In [0]:
data1 = pd.read_csv("test.csv",header=0)

C:\Users\keerthi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Understand the Data

See the No. row and columns

In [0]:
data.shape


(1687861, 23)

Display the columns

In [0]:
data.columns

Index(['sku', 'national_inv', 'lead_time', 'in_transit_qty',
       'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
       'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

Display the index

In [0]:
data.index

RangeIndex(start=0, stop=1687861, step=1)

See the top rows of the data

In [0]:
data[:3]

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1026827,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,No,No,No,Yes,No,No
1,1043384,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.99,0.99,0.0,No,No,No,Yes,No,No
2,1043696,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No


Shows a quick statistic summary of your data using describe

In [0]:
data.describe(include='all')

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
count,1687861,1.687860e+06,1.586967e+06,1.687860e+06,1.687860e+06,1.687860e+06,1.687860e+06,1.687860e+06,1.687860e+06,1.687860e+06,...,1.687860e+06,1.687860e+06,1.687860e+06,1.687860e+06,1687860,1687860,1687860,1687860,1687860,1687860
unique,1687861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,2,2,2,2,2
top,3284473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,No,No,Yes,No,No
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1300377,1687615,1484026,1626774,1687129,1676567
mean,NaN,4.961118e+02,7.872267e+00,4.405202e+01,1.781193e+02,3.449867e+02,5.063644e+02,5.592607e+01,1.750259e+02,3.417288e+02,...,2.043724e+00,-6.872059e+00,-6.437947e+00,6.264507e-01,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,2.961523e+04,7.056024e+00,1.342742e+03,5.026553e+03,9.795152e+03,1.437892e+04,1.928196e+03,5.192378e+03,9.613167e+03,...,2.360165e+02,2.655636e+01,2.584333e+01,3.372224e+01,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,-2.725600e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-9.900000e+01,-9.900000e+01,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,4.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,6.300000e-01,6.600000e-01,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.500000e+01,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,...,0.000000e+00,8.200000e-01,8.100000e-01,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,8.000000e+01,9.000000e+00,0.000000e+00,4.000000e+00,1.200000e+01,2.000000e+01,4.000000e+00,1.500000e+01,3.100000e+01,...,0.000000e+00,9.700000e-01,9.500000e-01,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN


Display data type of each variable

In [0]:
data.dtypes

sku                   object
national_inv         float64
lead_time            float64
in_transit_qty       float64
forecast_3_month     float64
forecast_6_month     float64
forecast_9_month     float64
sales_1_month        float64
sales_3_month        float64
sales_6_month        float64
sales_9_month        float64
min_bank             float64
potential_issue       object
pieces_past_due      float64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty         float64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

#### Observations

    sku is Categorical but is interpreted as int64 
    potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder are also categorical but is interpreted as object. 

#### Convert all the attributes to appropriate type

Data type conversion

    Using astype('category') to convert potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder attributes to categorical attributes.


In [0]:
for col in ['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
    data[col] = data[col].astype('category')
    

In [0]:
for col in ['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
    data1[col] = data1[col].astype('category')

Display data type of each variable

In [0]:
data.dtypes

sku                  category
national_inv          float64
lead_time             float64
in_transit_qty        float64
forecast_3_month      float64
forecast_6_month      float64
forecast_9_month      float64
sales_1_month         float64
sales_3_month         float64
sales_6_month         float64
sales_9_month         float64
min_bank              float64
potential_issue      category
pieces_past_due       float64
perf_6_month_avg      float64
perf_12_month_avg     float64
local_bo_qty          float64
deck_risk            category
oe_constraint        category
ppap_risk            category
stop_auto_buy        category
rev_stop             category
went_on_backorder    category
dtype: object

#### Delete sku attribute

In [0]:
np.size(np.unique(data.sku, return_counts=True)[0])

In [0]:
data.drop('sku', axis=1, inplace=True)

In [0]:
data1.drop('sku', axis=1, inplace=True)

#### Missing Data

    Missing value analysis and dropping the records with missing values

In [0]:
data.isnull().sum()

national_inv              1
lead_time            100894
in_transit_qty            1
forecast_3_month          1
forecast_6_month          1
forecast_9_month          1
sales_1_month             1
sales_3_month             1
sales_6_month             1
sales_9_month             1
min_bank                  1
potential_issue           1
pieces_past_due           1
perf_6_month_avg          1
perf_12_month_avg         1
local_bo_qty              1
deck_risk                 1
oe_constraint             1
ppap_risk                 1
stop_auto_buy             1
rev_stop                  1
went_on_backorder         1
dtype: int64

Observing the number of records before and after missing value records removal

In [0]:
print (data.shape)

(1687861, 22)


In [0]:
#Since the number of missing values is about 5%. For initial analysis we ignore all these records
data = data.dropna(axis=0)

In [0]:
data1 = data1.dropna(axis=0)

In [0]:
data.isnull().sum()
print(data.shape)

(1586967, 22)


#### Converting Categorical to Numeric

For some of the models all the independent attribute should be of type numeric and Linear Regression model is one among them.
But this data set has some categorial attributes.

'pandas.get_dummies' To convert convert categorical variable into dummy/indicator variables


In [0]:
print (data.columns)

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')


Creating dummy variables.

    If we have k levels in a category, then we create k-1 dummy variables as the last one would be redundant. So we use the parameter drop_first in pd.get_dummies function that drops the first level in each of the category


In [0]:
categorical_Attributes = data.select_dtypes(include=['category']).columns

In [0]:
categorical_Attributes = data1.select_dtypes(include=['category']).columns

In [0]:
data = pd.get_dummies(columns=categorical_Attributes, data=data, prefix=categorical_Attributes, prefix_sep="_",drop_first=True)

In [0]:
data1 = pd.get_dummies(columns=categorical_Attributes, data=data1, prefix=categorical_Attributes, prefix_sep="_",drop_first=True)

In [0]:
print (data.columns, data.shape)

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'pieces_past_due', 'perf_6_month_avg', 'perf_12_month_avg',
       'local_bo_qty', 'potential_issue_Yes', 'deck_risk_Yes',
       'oe_constraint_Yes', 'ppap_risk_Yes', 'stop_auto_buy_Yes',
       'rev_stop_Yes', 'went_on_backorder_Yes'],
      dtype='object') (1586967, 22)


#### Target attribute distribution

In [0]:
pd.value_counts(data['went_on_backorder_Yes'].values)

0    40000
1    11293
dtype: int64

In [0]:
pd.value_counts(data1['went_on_backorder_Yes'].values)

0    224747
1      2604
dtype: int64

#### Split the data in to train and test

sklearn.model_selection.train_test_split

    Split arrays or matrices into random train and test subsets

In [0]:
#Performing train test split on the data
#X, y = data.loc[:,data.columns!='went_on_backorder_Yes'].values, data.loc[:,'went_on_backorder_Yes'].values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


In [0]:
X_train = data.loc[:,data.columns!='went_on_backorder_Yes'].values
y_train = data.loc[:,'went_on_backorder_Yes'].values

In [0]:
X_test = data1.loc[:,data.columns!='went_on_backorder_Yes'].values
y_test = data1.loc[:,'went_on_backorder_Yes'].values

In [0]:
#To get the distribution in the target in train and test
print(pd.value_counts(y_train))
print(pd.value_counts(y_test))

0    40000
1    11293
dtype: int64
0    224747
1      2604
dtype: int64


#### Perceptron Model Building

In [0]:
perceptron_model = Sequential()

perceptron_model.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid', kernel_initializer='normal'))

In [0]:
perceptron_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
perceptron_model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5066 - acc: 0.7829
Epoch 2/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5061 - acc: 0.7830
Epoch 3/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5057 - acc: 0.7835
Epoch 4/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5053 - acc: 0.7835
Epoch 5/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5048 - acc: 0.7835
Epoch 6/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5044 - acc: 0.7839
Epoch 7/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5041 - acc: 0.7840
Epoch 8/50
51293/51293 [==============================] - 1s 25us/step - loss: 0.5036 - acc: 0.7839
Epoch 9/50
51293/51293 [==============================] - 1s 27us/step - loss: 0.5033 - acc: 0.7840
Epoch 10/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5030 - acc: 0.7845

#### Predictions

In [0]:
test_pred=perceptron_model.predict_classes(X_test)
train_pred=perceptron_model.predict_classes(X_train)

51293/51293 [==============================] - 1s 17us/step


#### Getting evaluation metrics and evaluating model performance

In [0]:
confusion_matrix_test = confusion_matrix(y_test, test_pred)
confusion_matrix_train = confusion_matrix(y_train, train_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

[[39707   293]
 [10673   620]]
[[216994   7753]
 [  1489   1115]]


#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [0]:
Accuracy_Train = (confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train)
print("Train TPR: ",TPR_Train)
print("Train Accuracy: ",Accuracy_Train)

Train TNR:  0.992675
Train TPR:  0.05490126627114141
Train Accuracy:  0.786208644454409


In [0]:
Accuracy_Test = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test)
print("Test TPR: ",TPR_Test)
print("Test Accuracy: ",Accuracy_Test)

Test TNR:  0.9655034327488242
Test TPR:  0.42818740399385563
Test Accuracy:  0.9593492001354733


#### MLP Model Building

In [0]:
mlp_model = Sequential()

mlp_model.add(Dense(12, input_dim=X_train.shape[1], activation='relu', kernel_initializer='glorot_normal'))
mlp_model.add(Dense(8, activation='relu', kernel_initializer='normal'))
mlp_model.add(Dense(1, activation='sigmoid', kernel_initializer='normal'))

In [0]:
mlp_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
mlp_model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
51293/51293 [==============================] - 2s 47us/step - loss: 0.5367 - acc: 0.7786
Epoch 2/50
51293/51293 [==============================] - 2s 31us/step - loss: 0.5009 - acc: 0.7837
Epoch 3/50
51293/51293 [==============================] - 2s 30us/step - loss: 0.4897 - acc: 0.7892
Epoch 4/50
51293/51293 [==============================] - 2s 30us/step - loss: 0.4781 - acc: 0.7935
Epoch 5/50
51293/51293 [==============================] - 2s 30us/step - loss: 0.4676 - acc: 0.7972
Epoch 6/50
51293/51293 [==============================] - 2s 31us/step - loss: 0.4613 - acc: 0.8005
Epoch 7/50
51293/51293 [==============================] - 2s 31us/step - loss: 0.4569 - acc: 0.8024
Epoch 8/50
51293/51293 [==============================] - 2s 31us/step - loss: 0.4530 - acc: 0.8043
Epoch 9/50
51293/51293 [==============================] - 2s 30us/step - loss: 0.4497 - acc: 0.8053
Epoch 10/50
51293/51293 [==============================] - 2s 30us/step - loss: 0.4464 - acc: 0.8066

#### Predictions

In [0]:
test_pred=mlp_model.predict_classes(X_test)
train_pred=mlp_model.predict_classes(X_train)

51293/51293 [==============================] - 1s 22us/step


#### Getting evaluation metrics and evaluating model performance

In [0]:
confusion_matrix_test = confusion_matrix(y_test, test_pred)
confusion_matrix_train = confusion_matrix(y_train, train_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

[[36728  3272]
 [ 4005  7288]]
[[223264   1483]
 [  2515     89]]


#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [0]:
Accuracy_Train=(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train= confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train= confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train)
print("Train TPR: ",TPR_Train)
print("Train Accuracy: ",Accuracy_Train)

Train TNR:  0.9182
Train TPR:  0.6453555299743203
Train Accuracy:  0.8581287895034411


In [0]:
Accuracy_Test=(confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test= confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test= confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test)
print("Test TPR: ",TPR_Test)
print("Test Accuracy: ",Accuracy_Test)

Test TNR:  0.9934014692075979
Test TPR:  0.03417818740399386
Test Accuracy:  0.9824148563234822


#### Derive new non-linear features using autoencoder

In [0]:
# The size of encoded and actual representations
encoding_dim = 16 
actual_dim = X_train.shape[1]

In [0]:
# Input placeholder
input_img = Input(shape=(actual_dim,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(actual_dim, activation='sigmoid')(encoded)

In [0]:
# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [0]:
print(autoencoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 17)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 16)                288       
_________________________________________________________________
dense_18 (Dense)             (None, 17)                289       
Total params: 577
Trainable params: 577
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
autoencoder.fit(X_train, X_train, epochs=30, batch_size=32)

Epoch 1/30
51293/51293 [==============================] - 4s 72us/step - loss: -0.1241
Epoch 2/30
51293/51293 [==============================] - 3s 58us/step - loss: -0.5562
Epoch 3/30
51293/51293 [==============================] - 3s 57us/step - loss: -0.6552
Epoch 4/30
51293/51293 [==============================] - 3s 57us/step - loss: -0.6984
Epoch 5/30
51293/51293 [==============================] - 3s 53us/step - loss: -0.7278
Epoch 6/30
51293/51293 [==============================] - 3s 56us/step - loss: -0.7467
Epoch 7/30
51293/51293 [==============================] - 3s 58us/step - loss: -0.7629
Epoch 8/30
51293/51293 [==============================] - 3s 59us/step - loss: -0.7770
Epoch 9/30
51293/51293 [==============================] - 3s 58us/step - loss: -0.7856
Epoch 10/30
51293/51293 [==============================] - 3s 58us/step - loss: -0.7918
Epoch 11/30
51293/51293 [==============================] - 3s 61us/step - loss: -0.7964
Epoch 12/30
51293/51293 [================

Create a separate encoder model

In [0]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [0]:
print(encoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 17)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 16)                288       
Total params: 288
Trainable params: 288
Non-trainable params: 0
_________________________________________________________________
None


#### derive new non-linear features

In [0]:
X_train_nonLinear_features = encoder.predict(X_train)
X_test_nonLinear_features = encoder.predict(X_test)

#### Combining new non-linear features to X_train and X_test respectively

In [0]:
X_train=np.concatenate((X_train, X_train_nonLinear_features), axis=1)
X_test=np.concatenate((X_test, X_test_nonLinear_features), axis=1)

#### Perceptron Model Building with both actual and non-linear features

In [0]:
perceptron_model = Sequential()

perceptron_model.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid', kernel_initializer='normal'))

In [0]:
perceptron_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
perceptron_model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
51293/51293 [==============================] - 2s 39us/step - loss: 0.5210 - acc: 0.7776
Epoch 2/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5084 - acc: 0.7824
Epoch 3/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5063 - acc: 0.7840
Epoch 4/50
51293/51293 [==============================] - 1s 27us/step - loss: 0.5048 - acc: 0.7843
Epoch 5/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5037 - acc: 0.7854
Epoch 6/50
51293/51293 [==============================] - 1s 27us/step - loss: 0.5025 - acc: 0.7859
Epoch 7/50
51293/51293 [==============================] - 1s 27us/step - loss: 0.5016 - acc: 0.7861
Epoch 8/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5008 - acc: 0.7863
Epoch 9/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.5000 - acc: 0.7867
Epoch 10/50
51293/51293 [==============================] - 1s 28us/step - loss: 0.4994 - acc: 0.7867

#### Predictions

In [0]:
test_pred=perceptron_model.predict_classes(X_test)
train_pred=perceptron_model.predict_classes(X_train)

51293/51293 [==============================] - 1s 17us/step


#### Getting evaluation metrics and evaluating model performance

In [0]:
confusion_matrix_test = confusion_matrix(y_test, test_pred)
confusion_matrix_train = confusion_matrix(y_train, train_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

[[39794   206]
 [10514   779]]
[[217329   7418]
 [  1455   1149]]


#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [0]:
Accuracy_Train=(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train= confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train= confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train)
print("Train TPR: ",TPR_Train)
print("Train Accuracy: ",Accuracy_Train)

Train TNR:  0.99485
Train TPR:  0.0689807845568051
Train Accuracy:  0.7910046205135204


In [0]:
Accuracy_Test=(confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test= confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test= confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test)
print("Test TPR: ",TPR_Test)
print("Test Accuracy: ",Accuracy_Test)

Test TNR:  0.9669939976951861
Test TPR:  0.4412442396313364
Test Accuracy:  0.9609722411601445
